In [1]:
import zarr
import xarray as xr
import numpy as np
import numpy as np
import dask
from pyseq import image_analysis as ia
from dask.distributed import Client
import torch
import joblib
from dask_jobqueue import SLURMCluster
import skimage
import time
import pickle
from os.path import exists, join
from joblib import Parallel, delayed
from joblib import parallel_backend
from dask.distributed import progress
from pathlib import Path

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
class snakemake():
    input  = ['/gpfs/commons/home/jsingh/results/20210323_4i4color/final_zarr/m1a.zarr', 
              '/gpfs/commons/home/jsingh/results/20210323_4i4color/masks/m1a.tiff']
    output = ['/gpfs/commons/home/jsingh/results/test_dict.pkl']

In [3]:
labels = skimage.io.imread(snakemake.input[1])
image_path = Path(snakemake.input[0])
im_name = image_path.stem
image = xr.open_zarr(image_path).to_array()
image = image.squeeze().drop_vars('variable').rename(im_name)

In [4]:
plane = image.sel(marker = 'LMN1b')

In [5]:
############  CPU PART #################

In [6]:
if torch.cuda.is_available() == False:

    def get_cluster(queue_name = 'pe2', log_dir=None):
        """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

            return cluster, client
        """

        cluster = SLURMCluster(
                    queue = queue_name, 
                    cores = 6 ,
                    memory = '60G',
                    walltime='1:00:00')
                    #extra=["--lifetime", "55m", "--lifetime-stagger", "4m"])
        client = Client(cluster, timeout="50s")

        return cluster, client

    cluster, client = get_cluster()

    def scale_cluster(count): 
        cluster.scale(count)
        return cluster.dashboard_link
    scale_cluster(5)

    #Way 3: Computing the same using dask compute without persisting Data
    val = np.max(labels)
    def get_pixels(lab):
        m = plane.values[labels == lab+1].mean()
        return m

    with parallel_backend('dask',scheduler_host=cluster.scheduler._address):
        results = Parallel(n_jobs=-1)(delayed(get_pixels)(lab) for lab in range(val))
        
    print(results)
    
    intensity_dict = {}
    intensity_dict.update({'m1a':results})
    with open(snakemake.output[0], 'wb') as f:
        pickle.dump(intensity_dict, f)
        
    client.close()
    cluster.close()
        
    
else:
    
    def get_mean_intensity(pl):
        result_ar = np.zeros(mx)
        tr = torch.from_numpy(pl)
        for r in range(mx):
            result_ar[r] = (tr[lab == r+1]).float().mean()
        return result_ar

    lab = torch.from_numpy(labels.astype('int'))
    mx = np.max(labels)
    pl = plane.values
    mean_int = get_mean_intensity(pl)
    
    print(mean_int)
    
        

    

[187.3165327731091, 405.14674195298375, 236.63436052631607, 418.5523536000002, 625.6932391891892, 472.22288666666276, 1368.0135612167287, 458.2640544776101, 1038.9491487499993, 864.4956617449665, 491.03020486486486, 1346.326246527777, 633.5295614035135, 308.8838207343412, 725.6246386956523, 323.667620261438, 545.439204347826, 1031.5620706140362, 574.3306688073394, 621.882193824228, 583.7760651459854, 474.7556858536584, 1297.6434507246381, 605.6318466666669, 447.72828660714293, 563.3206819301845, 674.6706222891564, 442.7224630769231, 587.4982739999999, 1007.2913143540674, 781.5368061598954, 483.98285447154336, 400.01251463414644, 744.6710952380946, 556.2058900584796, 867.0525188172044, 655.2405955752209, 862.1932053097363, 652.7372029154525, 1000.0350859223323, 616.7898627976197, 1110.5406411764698, 1253.970296875007, 1186.6857914529921, 575.2885360995858, 1049.9736830188715, 432.88534811625397, 496.5050701834864, 610.5017193277318, 798.9551809954768, 964.5459087962976, 799.079451965066

2022-08-06 19:15:15,606 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
